In [1]:
# Standard Library
import os
import glob
import copy
import random
import argparse

# Third-Party Libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, Subset

# Local Modules
import architecture
# from architecture import Model  # Make sure the class above is saved in mreegwavenet.py
import trainer
from utils import *

In [2]:
# Data information
num_channels = 19   # number of channels
num_classes  = 2    # number of classes
samp_freq    = 500  # Sampling frequency

# Architecture 
MP           = [10.0, 2.0]     # Multi-resolution list in seconds
feat_dim     = 32              # Feature dimension

# Training parameters
batch_size   = 32              # Number of images in each mini-batch
lr_rate      = 1e-03           # Learning-rate
patience     = 30              # patience 
tr_val_ratio = 0.7             # Train and Validation ratio 
class_ratio  = 2.0             # seizure nonseizure class ratio
num_epochs   = 10              # Number of sweeps over the dataset to train


num_timepoints = int(MP[0]*samp_freq)

In [3]:
data_dir = "test-data" 
test_data  = np.load(os.path.join(data_dir, "test-data.npy"))
test_label = np.load(os.path.join(data_dir, "test-label.npy"))

test_dataset = Dataset_tensor(test_data,  test_label)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [4]:
# Definition of Model
total_feat_len = int(np.sum([feat_dim * (MP[0] // s) for s in MP]))
print(f"total_feat_len: {total_feat_len}")

Model_mr = architecture.Model( n_chans=num_channels, 
                               n_classes=num_classes, 
                               feature_dim=feat_dim, 
                               Fs=samp_freq, 
                               SEC=MP,
                               tot_feat_len=total_feat_len).float()

# load pretrained model
model_path = "pre-trained-model/model_example.pt"
Model_mr.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
Model_mr.eval()  # Set to evaluation mode

total_feat_len: 192


Model(
  (temp_conv1): Conv1d(19, 19, kernel_size=(2,), stride=(2,), groups=19)
  (temp_conv2): Conv1d(19, 19, kernel_size=(2,), stride=(2,), groups=19)
  (temp_conv3): Conv1d(19, 19, kernel_size=(2,), stride=(2,), groups=19)
  (temp_conv4): Conv1d(19, 19, kernel_size=(2,), stride=(2,), groups=19)
  (temp_conv5): Conv1d(19, 19, kernel_size=(2,), stride=(2,), groups=19)
  (temp_conv6): Conv1d(19, 19, kernel_size=(2,), stride=(2,), groups=19)
  (chpool1): Sequential(
    (0): Conv1d(19, 32, kernel_size=(4,), stride=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv1d(32, 32, kernel_size=(4,), stride=(1,))
    (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
  (chpool2): Sequential(
    (0): Conv1d(19, 32, kernel_size=(4,), stride=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [5]:
save_dir = "results-pretrained"
os.makedirs(save_dir, exist_ok=True)

PRED, FEAT, PROB, TARGET = [], [], [], []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model_mr.to(device)

with torch.no_grad():
    for data, target in tqdm(test_loader):
            data = data.float().cuda() if torch.cuda.is_available() else data.float()
            target = target.cuda() if torch.cuda.is_available() else target
    
            pred, feat, prob = Model_mr(data)
            pred = torch.argmax(pred, dim=1).cpu().numpy()
    
            PRED.extend(pred)
            FEAT.append(feat.cpu())
            PROB.append(prob.cpu())
            TARGET.append(target.cpu())
        
PRED   = np.hstack(PRED)
FEAT   = np.vstack(FEAT)
PROB   = np.vstack(PROB)
TARGET = np.hstack(TARGET)     

print("Shape of pred:", PRED.shape)
print("Shape of feat:", FEAT.shape)
print("Shape of prob:", PROB.shape)
print("Shape of tgt:",  TARGET.shape)

np.save(os.path.join(save_dir, 'pred.npy'), PRED)
np.save(os.path.join(save_dir, 'feat.npy'), FEAT)
np.save(os.path.join(save_dir, 'prob.npy'), PROB)
np.save(os.path.join(save_dir, 'tgt.npy'), TARGET)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 146/146 [00:03<00:00, 44.77it/s]


Shape of pred: (4663,)
Shape of feat: (4663, 192)
Shape of prob: (4663, 2)
Shape of tgt: (4663,)


In [6]:
accuracy, precision, recall, specificity, f1, macro_f1, roc_auc = predict2perf(PRED, TARGET, PROB)

# Print results
print(f"Accuracy          : {accuracy:.4f}")
print(f"Precision         : {precision:.4f}")
print(f"Recall            : {recall:.4f}")
print(f"Specificity       : {specificity:.4f}")
print(f"F1 Score          : {f1:.4f}")
print(f"Macro F1          : {macro_f1:.4f}")
print(f"ROC-AUC           : {roc_auc:.4f}")

Accuracy          : 0.9292
Precision         : 0.0239
Recall            : 0.7273
Specificity       : 0.9297
F1 Score          : 0.0462
Macro F1          : 0.5047
ROC-AUC           : 0.9266
